In [31]:
import os

api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

llm_model = 'gpt-3.5-turbo'


In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [2]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short jokes {topic}"
)

model = ChatOpenAI()
output_parser = StrOutputParser()

C:\Users\Sahruz Riyad\AppData\Local\Temp\ipykernel_11744\508623184.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


In [3]:
chain = prompt | model | output_parser

In [4]:
chain.invoke({"topic": "cats"})

'Why was the cat sitting on the computer?\n\nIt wanted to keep an eye on the mouse!'

In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [6]:
vectorsotre = DocArrayInMemorySearch.from_texts(
    ["Reyad works at AI App Builder", "Cats likes milk"],
    embedding= OpenAIEmbeddings()
)

retriever = vectorsotre.as_retriever()

C:\Users\Sahruz Riyad\AppData\Local\Temp\ipykernel_11744\3473696739.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding= OpenAIEmbeddings()
d:\Work\Testing\myenv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [7]:
retriever.get_relevant_documents("where did reyad work?")

C:\Users\Sahruz Riyad\AppData\Local\Temp\ipykernel_11744\1079775495.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("where did reyad work?")


[Document(metadata={}, page_content='Reyad works at AI App Builder'),
 Document(metadata={}, page_content='Cats likes milk')]

In [8]:
retriever.get_relevant_documents("what does bear likes to eat?")

[Document(metadata={}, page_content='Cats likes milk'),
 Document(metadata={}, page_content='Reyad works at AI App Builder')]

In [9]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template) # from_template Use when: You need a single message with placeholders for formatting.

In [10]:
from langchain.schema.runnable import RunnableMap

In [11]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x['question']),
    "question": lambda x: x['question'] 
}) | prompt | model | output_parser

In [12]:
chain.invoke({"question": "where did reyad work?"})

'Reyad works at AI App Builder.'

In [13]:
chain.invoke({"question": "who is the president of america?"})

"I'm sorry, but based on the provided context, there is no information about the current president of America."

In [14]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x['question']),
    "question": lambda x: x['question'] 
})

In [15]:
inputs.invoke({"question": "where did reyad work?"})

{'context': [Document(metadata={}, page_content='Reyad works at AI App Builder'),
  Document(metadata={}, page_content='Cats likes milk')],
 'question': 'where did reyad work?'}

In [16]:
inputs.invoke({"question": "what does cats eat?"})


{'context': [Document(metadata={}, page_content='Cats likes milk'),
  Document(metadata={}, page_content='Reyad works at AI App Builder')],
 'question': 'what does cats eat?'}

In [17]:
inputs.invoke({"question": "who is the president of america?"})


{'context': [Document(metadata={}, page_content='Reyad works at AI App Builder'),
  Document(metadata={}, page_content='Cats likes milk')],
 'question': 'who is the president of america?'}

🔹 bind() in LangChain
In LangChain, .bind() is used to pre-configure a chain or component with fixed values. This is useful when you want to set default parameters that will always be used when calling the chain, without needing to pass them every time.

In [18]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [19]:
prompt = ChatPromptTemplate.from_messages( # from_messages Use when: You need multiple messages (e.g., system instructions, user input, assistant responses).
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [20]:
runnable = prompt | model

In [21]:
response = runnable.invoke({"input": "what is the weather in sf"})

In [22]:
import json

In [23]:
args = json.loads(response.additional_kwargs['function_call']['arguments'])
print(args)

{'airport_code': 'SFO'}


In [24]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [25]:
model = model.bind(functions=functions)

In [26]:
runnable = prompt | model

In [27]:
response = runnable.invoke({"input": "how did the patriots do yesterday?"})

In [28]:
print(response)

content='' additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 99, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None} id='run-1015db0c-6310-40e9-b25a-60e75689a72e-0'


In [29]:
args = json.loads(response.additional_kwargs['function_call']['arguments'])
print(args)
fn_name = function_name = response.additional_kwargs['function_call']['name']
print(fn_name)

{'team_name': 'patriots'}
sports_search


Fallbacks

In [30]:
from langchain.llms import OpenAI
import json

In [32]:
simple_model = OpenAI(
    temperature=0,
    max_tokens= 3000,
    model= "gpt-3.5-turbo-instruct"
)

simple_chain = simple_model | json.loads

C:\Users\Sahruz Riyad\AppData\Local\Temp\ipykernel_11744\3735906933.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  simple_model = OpenAI(


In [34]:
challenge = "Write three poems in a json blob, where each peom is json blob of title, author, and first line"

In [35]:
simple_model.invoke(challenge)

'\n\n{\n    "title": "The Road Not Taken",\n    "author": "Robert Frost",\n    "first_line": "Two roads diverged in a yellow wood," \n}\n\n{\n    "title": "Still I Rise",\n    "author": "Maya Angelou",\n    "first_line": "You may write me down in history" \n}\n\n{\n    "title": "The Love Song of J. Alfred Prufrock",\n    "author": "T.S. Eliot",\n    "first_line": "Let us go then, you and I," \n}'

In [ ]:
simple_chain.invoke(challenge)

In [42]:
new_model = ChatOpenAI(template=0,model_name = 'gpt-4.0-turbo')
new_model_chain = model | StrOutputParser()

WARNING! template is not default parameter.
                    template was transferred to model_kwargs.
                    Please confirm that template is what you intended.


In [43]:
new_model_chain.invoke(challenge)

'```json\n{\n  "poem1": {\n    "title": "The Rose",\n    "author": "Emily Dickinson",\n    "first_line": "A rose is a rose is a rose"\n  },\n  "poem2": {\n    "title": "The Road Not Taken",\n    "author": "Robert Frost",\n    "first_line": "Two roads diverged in a yellow wood"\n  },\n  "poem3": {\n    "title": "Hope is the Thing with Feathers",\n    "author": "Emily Dickinson",\n    "first_line": "Hope is the thing with feathers"\n  }\n}\n```'

In [44]:
final_chain = simple_chain.with_fallbacks([new_model_chain])

In [45]:
final_chain.invoke(challenge)

'```json\n{\n  "poems": [\n    {\n      "title": "The Rose",\n      "author": "Emily Dickinson",\n      "first_line": "A rose is a rose is a rose"\n    },\n    {\n      "title": "The Road Not Taken",\n      "author": "Robert Frost",\n      "first_line": "Two roads diverged in a yellow wood"\n    },\n    {\n      "title": "Hope is the Thing with Feathers",\n      "author": "Emily Dickinson",\n      "first_line": "Hope is the thing with feathers"\n    }\n  ]\n}\n```'

Interface:
In LangChain, an interface is an abstraction that defines how components interact. It allows for flexibility and modularity by enabling different implementations to work interchangeably.

In [46]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [47]:
chain.invoke({"topic": "cats"})

'Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!'

In [48]:
chain.batch([{"topic": "cats"}, {"topic": "dogs"}])

['Why was the cat sitting on the computer?\n\nHe wanted to keep an eye on the mouse!',
 "Why did the dog sit in the shade? Because he didn't want to be a hot dog!"]

In [49]:
for t in chain.stream({"topic": "frogs"}):
    print(t)


Why
 are
 frogs
 so
 happy
?
 Because
 they
 eat
 whatever
 bugs
 them
!



All of the interface method has asynchronous. add a before the method call

In [52]:
response = await chain.ainvoke({"topic": "cows"})
response

'Why did the cow go to outer space? Because it wanted to see the moooon!'